In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import copy

from psm_utils.io.peptide_record import peprec_to_proforma
from psm_utils.psm import PSM
from psm_utils.psm_list import PSMList
from psm_utils.io import write_file
from deeplc import FeatExtractor

In [3]:
# Read the CSV file into a DataFrame
df = pd.read_csv('/home/emmy/Notebooks2/output/MQpeprec.csv')

# Replace "Citrullination" with "Deamidation" in all columns
df = df.replace("Citrullination", "Deamidation")

# Write the DataFrame back to a CSV file
df.to_csv('/home/emmy/Notebooks2/output/MQpeprec_1.csv', index=False)

In [7]:
df[df['modifications'].str.contains('Citrullination', na=False)].head()

,Unnamed: 0,sequence,modifications,charge,tr
14408,2590916,AAGGAPLSCFICGGGIGRGK,9|Carbamidomethyl|12|Carbamidomethyl|18|Citrul...,2,457.243285
14409,2590917,AAGGAPLSCFICGGGIGRGK,9|Carbamidomethyl|12|Carbamidomethyl|18|Citrul...,3,545.810620
14984,2590930,AAHGRGALAEAAR,5|Citrullination,2,382.786007
14985,2590939,AAHGRGALAEAAR,5|Citrullination,3,463.892010
20120,2590957,AAYQAAIRGVGSAQSR,8|Citrullination|16|Citrullination,2,414.281695


In [8]:
df['modifications'] = df['modifications'].str.replace('Citrullination', 'Deamidation')

# Verify the replacement
df[df['modifications'].str.contains('Citrullination', na=False)].head()

,Unnamed: 0,sequence,modifications,charge,tr


In [9]:
# Write the DataFrame back to a CSV file
df.to_csv('/home/emmy/Notebooks2/output/MQpeprec_1.csv', index=False)

In [2]:
infile = pd.read_csv("/home/emmy/Notebooks2/output/MQpeprec_1.csv") #hier eigen aligned csv file aanroepen

infile.head()
psm_list = [] #psm_list is type object 
for idx,row in infile.iterrows():
    seq = row["sequence"]
    charge = row["charge"]  # Get the charge from the row
    mod = row["modifications"]
    
    peptidoform = f"{seq}/{charge}"
    psm_list.append(PSM(peptidoform=peprec_to_proforma(peptidoform,mod),spectrum_id=idx))
    
psm_list = PSMList(psm_list=psm_list)

feat_extractor = FeatExtractor()
matrices = feat_extractor.encode_atoms(psm_list, list(range(len(psm_list))), predict_ccs=True)





In [3]:
data = pd.read_csv("/home/emmy/Notebooks2/output/MQpeprec_1.csv") #reading in the data
ccs_df = data[['tr']]

In [4]:
ccs_df.head()

,tr
0,409.181586
1,675.752631
2,593.085242
3,682.043900
4,492.912110


In [5]:
matrix = np.stack(list(matrices["matrix"].values()))
matrix_all = np.stack(list(matrices["matrix_all"].values()))

In [5]:
# Reshape 'matrix' to 2D and save to a CSV file
#np.savetxt("matrix2.csv", matrix.reshape(matrix.shape[0], -1), delimiter=",")

# Reshape 'matrix_all' to 2D and save to a CSV file
#np.savetxt("matrix_all2.csv", matrix_all.reshape(matrix_all.shape[0], -1), delimiter=",")

In [18]:
matrix_all.shape

(1148760, 12)

In [19]:
matrix.shape

(1148760, 60, 6)

In [6]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
matrix_train, matrix_test, matrix_all_train, matrix_all_test, ccs_train, ccs_test = train_test_split(matrix, matrix_all, ccs_df, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets if needed
matrix_train, matrix_val, matrix_all_train, matrix_all_val, ccs_train, ccs_val = train_test_split(matrix_train, matrix_all_train, ccs_train, test_size=0.1, random_state=42)

In [7]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('/home/emmy/Notebooks2/output/MQpeprec_1.csv')

# Split the 'modifications' column by ';' (or the appropriate delimiter)
modifications = df['modifications'].str.split(';', expand=True).stack()

# Split each modification by '|' to separate the position from the modification itself
modifications = modifications.str.split('|').str[1]

# Count the number of different modifications
modification_counts = modifications.value_counts()

# Print the counts
print(modification_counts)

Carbamidomethyl      174485
Oxidation            145471
Acetyl                11582
Deamidation            8461
Methyl                  671
Dimethyl                667
GlyGly                  490
Hydroxyproline          478
Trimethyl               395
Gluratylation           391
Succinyl                383
Butyryl                 362
hydroxyisobutyryl       355
Crotonyl                312
Formyl                  285
Malonyl                 260
Biotin                  194
Nitro                   189
HexNAc                  169
Propionyl               156
Phospho                 132
dtype: int64


In [18]:
# Create a boolean mask for rows where the 'modifications' column contains "Carbamidomethyl"
mask = df['modifications'].str.contains('Phospho', na=False)

# Get the indices of the rows where the mask is True
indices = df[mask].index.tolist()

# Print the indices
print(indices)

#moet wel +2 doen om zelfde getal te hebben als in peprec_1.csv

[14714, 14715, 27652, 27653, 30992, 45407, 47870, 61143, 68014, 75003, 77267, 94341, 94342, 94343, 94344, 94345, 94346, 94347, 102601, 103563, 104873, 112979, 132928, 138766, 139384, 139715, 162617, 162618, 162619, 162620, 162621, 162622, 251515, 259784, 259785, 268936, 270147, 281356, 286713, 297612, 301918, 311961, 330116, 330325, 330326, 336075, 344269, 344270, 346681, 348397, 348398, 348399, 360090, 360213, 360214, 361679, 361680, 398532, 419337, 426720, 446930, 446931, 595615, 595616, 599022, 607802, 610509, 613182, 613183, 615305, 626986, 629677, 663904, 678893, 680529, 689353, 692268, 710023, 710024, 710025, 710026, 710027, 710028, 711313, 711314, 713650, 714369, 717658, 717659, 803292, 804882, 897493, 898605, 902452, 933654, 950709, 958803, 960598, 960599, 960600, 960601, 960602, 960603, 960604, 960605, 965125, 975719, 978477, 983735, 983736, 994542, 1010225, 1012066, 1014218, 1014219, 1023759, 1033864, 1040737, 1042604, 1052316, 1052317, 1059285, 1076481, 1097245, 1100516, 110

In [19]:
# Create a new list where every number is incremented by 2
new_indices = [idx + 2 for idx in indices]

print(new_indices)


[14716, 14717, 27654, 27655, 30994, 45409, 47872, 61145, 68016, 75005, 77269, 94343, 94344, 94345, 94346, 94347, 94348, 94349, 102603, 103565, 104875, 112981, 132930, 138768, 139386, 139717, 162619, 162620, 162621, 162622, 162623, 162624, 251517, 259786, 259787, 268938, 270149, 281358, 286715, 297614, 301920, 311963, 330118, 330327, 330328, 336077, 344271, 344272, 346683, 348399, 348400, 348401, 360092, 360215, 360216, 361681, 361682, 398534, 419339, 426722, 446932, 446933, 595617, 595618, 599024, 607804, 610511, 613184, 613185, 615307, 626988, 629679, 663906, 678895, 680531, 689355, 692270, 710025, 710026, 710027, 710028, 710029, 710030, 711315, 711316, 713652, 714371, 717660, 717661, 803294, 804884, 897495, 898607, 902454, 933656, 950711, 958805, 960600, 960601, 960602, 960603, 960604, 960605, 960606, 960607, 965127, 975721, 978479, 983737, 983738, 994544, 1010227, 1012068, 1014220, 1014221, 1023761, 1033866, 1040739, 1042606, 1052318, 1052319, 1059287, 1076483, 1097247, 1100518, 110

In [8]:
ccs_test

,tr
129919,420.248133
1129049,441.017200
161312,550.032030
15185,501.311247
939427,520.231105
...,...
752472,428.058479
208958,519.166400
419260,487.869235
552329,598.787710


In [8]:
# Read the CSV file into a DataFrame
df = pd.read_csv('ccs_test_model_combined_mod.csv')

# Set the index of df to the index of ccs_test
df.index = ccs_test.index

# Save df back to a CSV file
#df.to_csv('ccs_test_model_rnn_mod_gru.csv', index=True)

In [9]:
df.head()

,CCS,Model_combined_predictions_mod
129919,420.248133,418.19495
1129049,441.017200,436.70187
161312,550.032030,580.59080
15185,501.311247,511.33180
939427,520.231105,505.11752


In [10]:
# Save df back to a CSV file
df.to_csv('ccs_test_model_combined_mod.csv', index=True)

In [ ]:
# Or, if you don't know the column name, you can use iloc to access the first column
valid_indices = [idx for idx in phospho_indices if idx in df["Indexnr"]]